<a href="https://colab.research.google.com/github/isakhammer/numerical-mathematics-project-2/blob/master/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Main Concept**

Compare Integrators, and ODE-networks vs Hamilton Networks. 


**Generate Synthetic Input Data**


Synthetic data is "any production data applicable to a given situation that are not obtained by direct measurement"


In [ ]:



def RK4(x0, t0, T, f, h):
  N = (T - t0)/h
  
  xi = x0
  ti = t0
  while ti < T:
    k1 = f(xi, ti)
    k2 = f(xi + h*k1*0.5, ti + h*0.5)
    k3 = f(xi + h*k2*0.5, ti + h*0.5)
    k4 = f(xi + h*k3, ti + h)
    xi += (h/6)*(k1 + k2*2 + k3*2 +k4) 

"""
Symplectic Euler Method for 2 dim.

Notation:
p - Generalized Coordinates
q - Conjugate momenta
V -  Potential energy function
T - Kinetic energy function

Input:
 p0: (2,1) numpy array
 q0: (2,1) numpy array
 t0: Inital time
 T: Terminal time
 dV: Function dV(p,q)/dp
 dT: Function dT(p,q)/dq
"""

def symplectic_euler_method(p0, q0, t0, T, h, dV, dT)
  
  N = int(T/h):
  q = np.zeros((q0.shape[0], N))
  p = np.zeros((p0.shape[0], N))
  t = np.zeros((1, N))

  q[0] = q0
  p[0] = p0
  t[0] = t0
  for i in range(N):
    q[i] = q[i-1] + h*dT(q[i-1])
    p[i] = p[i-1] +  h*dT(p[i-1])
    t[i] = t[i-1] + h

  return p,q,t





SyntaxError: ignored